## A demo using Hydrogen Hamiltonian with GPT-QE.

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from gqe.mingpt.utils import set_seed

set_seed(3407)

In [2]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, generate_molecule
from gqe.common.initializer import HFStateInitializer

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
molecule = generate_molecule("Li", "H", 3.0, "sto-3g", bravyi_kitaev=False)
nqubit = 10

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=2)
pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                  [0.003125, -0.003125, 0.00625, -0.00625, 0.0125, -0.0125, 0.025, -0.025, 0.05, -0.05, 0.1, -0.1])


converged SCF energy = -7.71082990021723
Starting to parse FermionOperator using 10 qubits...

Operator t:  -5.951285666047443 [] +
-0.01525756739418034 [X0 X1 Y2 Y3] +
-0.018349876887035498 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Y9] +
-0.018349876887035495 [X0 X1 X3 Z4 Z5 Z6 Z7 X8] +
-0.005458646052827126 [X0 X1 Y4 Y5] +
-0.005458646052827126 [X0 X1 Y6 Y7] +
-0.030398420879386975 [X0 X1 Y8 Y9] +
0.01525756739418034 [X0 Y1 Y2 X3] +
0.018349876887035498 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 X9] +
-0.018349876887035495 [X0 Y1 Y3 Z4 Z5 Z6 Z7 X8] +
0.005458646052827126 [X0 Y1 Y4 X5] +
0.005458646052827126 [X0 Y1 Y6 X7] +
0.030398420879386975 [X0 Y1 Y8 X9] +
0.039551719621254526 [X0 Z1 X2] +
-0.012496233638660552 [X0 Z1 X2 X3 Z4 Z5 Z6 Z7 Z8 X9] +
-0.012496233638660552 [X0 Z1 X2 Y3 Z4 Z5 Z6 Z7 Z8 Y9] +
-0.003663428542883019 [X0 Z1 X2 Z3] +
-0.002979380650007077 [X0 Z1 X2 Z4] +
-0.009039934734978024 [X0 Z1 X2 Z5] +
-0.002979380650007077 [X0 Z1 X2 Z6] +
-0.009039934734978024 [X0 Z1 X2 Z7] +
-0.0012148728915797238 

## FCI energy by diagonalization

In [3]:
from qwrapper.hamiltonian import compute_ground_state

print(compute_ground_state(hamiltonian))
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(10, [], "qulacs")))

-7.798504222541834
hf state: -7.710829900217204


## Setup for GPT

In [4]:
# create a GPT instance
from gqe.mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size = cost.vocab_size()
model_config.block_size = 4
model_config.n_gates = 8 # The number of gates for each circuit
model_config.temperature = 10  # Each gate is generated with probability exp(-temperature * logit)
model_config.embd_pdrop = 0.2
model_config.resid_pdrop = 0.2
model_config.attn_pdrop = 0.2
model_config.energy_scaling = 5
model = GPT(model_config, cost)

number of parameters: 85.68M


In [5]:
# create a Trainer object
from gqe.mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
train_config.max_iters = 500
train_config.num_workers = 0
train_config.n_samples = 5
trainer = Trainer(train_config, model)

running on device cpu


In [6]:
def batch_end_callback(trainer):
    if trainer.iter_num % 1 == 0:
        print(
            f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f} temperature: {model.temperature}")
        model.temperature += 0.01


trainer.set_callback('on_batch_end', batch_end_callback)
trainer.run()
torch.save(model.state_dict(), '../saved_models/gptqe_test_2')

tensor([[801, 202, 801,  57, 719, 720, 670, 263],
        [675, 347, 249, 336,  38, 109,  25, 265],
        [131, 675, 607, 474, 180, 305, 647, 352],
        [438,  23, 364, 537, 290, 782,  94, 455],
        [336, 619, 490, 398, 487, 424, 656,  72]])


RuntimeError: Size does not match at dimension 1 expected index [5, 8, 1] to be smaller than self [5, 4, 804] apart from dimension 2

In [ ]:
indices, logits = model.generate(torch.tensor([[0]]), model_config.n_gates)
print(cost.energy(indices))

In [ ]:
model.temperature = 20
model.load_state_dict(torch.load('../saved_models/gptqe_test_2'))
indices, logits = model.generate(torch.zeros(30, 1, dtype=torch.int), model_config.n_gates)
cost.sequence.tool = "qiskit"
index = torch.argmin(cost.energy(indices)).item()

In [ ]:
target = indices.numpy()[index]
print(cost.energy(torch.tensor([target])))
print(target)
cost.sequence._get_circuit(target).qc.draw(output="mpl", plot_barriers=True)